In [ ]:
!git clone https://github.com/joy98/video-lecture-segmentation.git

Cloning into 'video-lecture-segmentation'...
remote: Enumerating objects: 624, done.
remote: Total 624 (delta 0), reused 0 (delta 0), pack-reused 624
Receiving objects: 100% (624/624), 14.69 MiB | 21.21 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import re
from nltk import*
import numpy as np
import pandas as pd
import tensorflow_hub as hub
'''
    NOUN PHRASE EXTRACTION
'''
freqW=stopwords.words('english')      #list containing stop words

# Defining a grammar & Parser
NP = "NP: {<NN.*><VB.?>+<NN.*>}"
chunker = RegexpParser(NP)

## Noun Phrase Extraction ###

def get_continuous_chunks(text, chunk_func=ne_chunk):
    ptxt=''
    for w in text.split(' '):  #removing stop words
        if w not in freqW:
            ptxt+=w+' '
    text=ptxt.strip()
    chunked = chunk_func(pos_tag(word_tokenize(text)))
    continuous_chunk = []
    current_chunk = []

    for subtree in chunked:
        if type(subtree) == Tree:
            current_chunk.append(" ".join([token for token, pos in subtree.leaves()]))
        elif current_chunk:
            named_entity = " ".join(current_chunk)
            if named_entity not in continuous_chunk:
                continuous_chunk.append(named_entity)
                current_chunk = []
        else:
            continue

    return continuous_chunk

In [ ]:
!pip install flair

     |████████████████████████████████| 286kB 16.0MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
     |████████████████████████████████| 983kB 33.1MB/s 
     |████████████████████████████████| 776.8MB 22kB/s 
     |████████████████████████████████| 19.7MB 1.3MB/s 
     |████████████████████████████████| 2.0MB 41.6MB/s 
     |████████████████████████████████| 798kB 59.6MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.2MB 43.5MB/s 
     |████████████████████████████████| 61kB 8.0MB/s 
     |████████████████████████████████| 3.2MB 38.8MB/s 
     |████████████████████████████████| 890kB 44.3MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-cp37-none-any.whl size=9693 sha256=9ea507f6ff163d4144ebee9c908ee5ff59ee1ab513f0bd0cccc578f6b6c86e0c
  Stored in directory: /root/.cache/pip/wheels/81/d0/d7/d9983facc6f2775411803e0e2d30ebf98efbf2fc6e57701e09
Succes

In [ ]:
!pip install bert-tensorflow

     |████████████████████████████████| 71kB 6.9MB/s 


In [ ]:
from flair.embeddings import TransformerWordEmbeddings

In [ ]:
import nltk
nltk.download('punkt')
  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
path=r"/content/video-lecture-segmentation/ALV_srt"
tmp=[]
for i in range(1,100):
    tmp.append(str('%04d'%i))
files=[open(path+'/'+i+'.srt',encoding="utf8") for i in tmp]
files=[i.read() for i in files]
files=[word_tokenize(i) for i in files]

In [ ]:
#regex for words and time in the text
regex1 = re.compile("([a-zA-Z]+)")
regex2 = re.compile("\d\d")

In [ ]:
words_list=[]   #word_list contains the words from the input text
words_time=[]   #words_time contains the time corresponding to each words from words_list

start_time,end_time=None,None
for file in files:
    tmp=[]
    ptm=[]
    for word in file:
        if regex1.match(word):
            k=regex1.match(word).span()
            tmp.append(word[k[0]:k[1]])
            ptm.append(start_time)
        elif regex2.match(word):
            if start_time==None and end_time==None:
                start_time=word
            elif end_time==None:
                end_time=word
            else:
                start_time=word
                end_time=None
    words_list.append(tmp)
    words_time.append(ptm)

In [ ]:
#processed word contains the words from words_list after removal of stop words
processed_word=[]
for ws in words_list:
    tmp=[]
    for word in ws:
            tmp.append(word.lower())
    processed_word.append(tmp)
words_list=processed_word

In [ ]:
#BERT model
model = TransformerWordEmbeddings('bert-base-uncased')

#using pretrained model

In [ ]:
from flair.data import Sentence

In [ ]:

#calculate_cosine_similarity function computes cosine of two vector passed in the argument

def calculate_cosine_similarity(vec1,vec2):

    l1=len(vec1)
    l2=len(vec2)
    l3=min(l1,l2);

    vec1=numpy.asarray(vec1).flatten()[0:l3]
    vec2=numpy.asarray(vec2).flatten()[0:l3]
    csim = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    if np.isnan(np.sum(csim)):
        return 0
    #print(csim)
    return csim

#vectorize function takes mean of all the words in noun phrases
def vectorize(vectors):
    #print(vectors)
    word_vecs=[]
    v2=[]
    for vector in vectors:
        sentence=""
        for word in vector.split(" "):
            word_vecs.append(word)
            sentence+=word+" "
        sentence1=Sentence(sentence)
            #print(sentence1)
        model.embed(sentence1)
        for token in sentence1:
              #print(token.embedding)
            v2.append(token.embedding.numpy())
              #print(token.embedding)
            #print(v2)
  
    #v2 = [t.numpy() for t in my_list_of_tensors]
    #print(shape(v2))
    #ans = np.mean(v2 , axis=0)
    return v2

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
len(words_list)

99

In [ ]:
'''
Creating the frames required for cosine similarity comparision
'''
window_size=1080

step_size=window_size//6
window_time_start=[]
window_time_end=[]
window_cue=[]

#ith denotes  transcript file number

for ith in range(99):                
    tmp=[]
    start=[]
    end=[]
    for i in range(0, len(words_list[ith]),step_size):
        if i + window_size < len(words_list[ith]):
            start.append(words_time[ith][i])
            end.append(words_time[ith][i+window_size])
            strw=""
            for w in words_list[ith][i:i+window_size]:
                strw+=w+' '
            tmp.append(get_continuous_chunks(strw,chunker.parse))
    window_time_start.append(start)
    window_time_end.append(end)
    window_cue.append(tmp)


In [ ]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
#path for ground truth value
path2=r'/content/video-lecture-segmentation/ALV_srt_GT'

In [ ]:
#reading all the ground truth files

tmp2=[]
for i in range(1,101):
    tmp2.append(str('%04d'%i))
    
groundTruth=[open(path2+'/'+i+'.txt',encoding="utf8") for i in tmp2]
groundTruth=[i.read() for i in groundTruth]
groundTruth=[i.split('\n') for i in groundTruth]

In [ ]:
# gtv contains timestamp from ground truth
gtval=[]
for i in groundTruth:
    tmp=[]
    for j in i:
        if len(j)>1:
            tmp.append(j.split('\t'))
    gtval.append(tmp)
gtv=[]
for i in gtval:
    tmp=[]
    for j in i:
        if len(j)>1:
            tmp.append(j[2])
    gtv.append(tmp)

In [ ]:
#score_list contains cosine similarity of ith and (i+1)th window
score_list=[]
k=0
for i in window_cue:
    tmp=[]
    k+=1
    print(k)
    for j in range(0,len(i)-1):
        tmp.append(calculate_cosine_similarity(vectorize(i[j]),vectorize(i[j+1])))
    score_list.append(tmp)
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
94
95
96


In [ ]:
#depth_list contains minimas of score_list
#depth_time_list contains time for each depth

depth_list=[]
depth_time_list=[]
z=0
for j,k in zip(score_list,window_time_start):
    tmp_list=[]
    tmp_time=[]
    z+=1
    #if(z==6):
     # break

    for i in range(1,len(j)-1):
        if(j[i-1]>j[i] and j[i]<j[i+1]):
            tmp_list.append(j[i+1]+j[i-1]-j[i]-j[i])
            tmp_time.append(k[i])
    depth_list.append(tmp_list)
    depth_time_list.append(tmp_time)

NameError: ignored

In [ ]:
depth_time_list

In [ ]:
#function diff calculate difference between two time with t1,t2 as timestamp and tsec as permitable difference
def diff(t1,t2,tsec):
    if len(t1) !=12 or len(t2)!=12:
        return False
    hh1,mm1,ss1=map(int,t1[0:-4].split(':'))
    hh2,mm2,ss2=map(int,t2[0:-4].split(':'))
    dh=abs(hh1-hh2)
    dm=abs(mm1-mm2)
    ds=abs(ss1-ss2)
    if dh*60*60+dm*60+ds>=tsec:
        return False
    return True

In [ ]:
#sc1,sc2,sc3,sc4 denotes score for 30sec,45sec,60sec and 90 sec permitable difference

sc1,sc2,sc3,sc4=[],[],[],[]

for i in range(5):
    t1,t2,t3,t4=0,0,0,0
    s1,s2,s3,s4=set({}),set({}),set({}),set({})
    for j in depth_time_list[i]:
        for k in range(20):
            #print(j,gtv[i][k])
            if diff(j,gtv[i][k],30) and gtv[i][k] not in s1:
                t1+=1
                s1.add(gtv[i][k])
            if diff(j,gtv[i][k],45) and gtv[i][k] not in s2:
                t2+=1
                s2.add(gtv[i][k])
            if diff(j,gtv[i][k],60) and gtv[i][k] not in s3:
                t3+=1
                s3.add(gtv[i][k])
            if diff(j,gtv[i][k],90) and gtv[i][k] not in s4:
                t4+=1
                s4.add(gtv[i][k])
    sc1.append(t1)
    sc2.append(t2)
    sc3.append(t3)
    sc4.append(t4)

In [ ]:
# Precision= score/total_no_of_predicted_boundaries
# Recall=  score/actual_number_of_ground_truth_value
# F-Score= 2*Precision*Recall/(Precision+Recall)

precision1=[sc1[i]/len(depth_time_list[i]) for i in range(100)]
precision2=[sc2[i]/len(depth_time_list[i]) for i in range(100)]
precision3=[sc3[i]/len(depth_time_list[i]) for i in range(100)]
precision4=[sc4[i]/len(depth_time_list[i]) for i in range(100)]
recall1=[sc1[i]/20 for i in range(100)]
recall2=[sc2[i]/20 for i in range(100)]
recall3=[sc3[i]/20 for i in range(100)]
recall4=[sc4[i]/20 for i in range(100)]
fscore1,fscore2,fscore3,fscore4=[],[],[],[]
for i in range(5):
    if sc1[i]==0:
        fscore1.append(0)
    else:
        fscore1.append((2*precision1[i]*recall1[i])/(precision1[i]+recall1[i]))
    if sc2[i]==0:
        fscore2.append(0)
    else:
        fscore2.append((2*precision2[i]*recall2[i])/(precision2[i]+recall2[i]))
    if sc3[i]==0:
        fscore3.append(0)
    else:
        fscore3.append((2*precision3[i]*recall3[i])/(precision3[i]+recall3[i]))
    if sc4[i]==0:
        fscore4.append(0)
    else:
        fscore4.append((2*precision4[i]*recall4[i])/(precision4[i]+recall4[i]))

In [ ]:
# avg_30,avg_45,avg_60,avg_90 average score of 100 files

avg_30=sum(fscore1)/100
avg_45=sum(fscore2)/100
avg_60=sum(fscore3)/100
avg_90=sum(fscore4)/100
recall_30=sum(recall1)/100
recall_45=sum(recall2)/100
recall_60=sum(recall3)/100
recall_90=sum(recall4)/100

In [ ]:
print(avg_30,avg_45,avg_60,avg_90)
print(recall_30,recall_45,recall_60,recall_90)